In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Load Data set

In [3]:
df = pd.read_csv(r'C:\Users\HP\Downloads\titanic (1).csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
#Select 'Pclass','SibSp','Parch','Fare' columns as data. Select ‘Survived’ column as label.

X = df[['Pclass','SibSp','Parch','Fare']]
y = df.Survived

# Model Building

In [6]:
# Apply SVM to classify the data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [7]:
from sklearn.svm import SVC
model_svc = SVC(kernel='linear')
model_svc.fit(X_train,y_train)

y_pred = model_svc.predict(X_test)

print('Accuracy Score:\n',accuracy_score(y_test, y_pred))

Accuracy Score:
 0.7653631284916201


In [8]:
# Use 5-Fold Cross Validation to generalise the model accuracy.

from sklearn.model_selection import cross_val_score,KFold

kf=KFold(n_splits=5)
score=cross_val_score(model_svc,X,y,cv=kf)

print("Cross Validation Scores are: {}\n".format(score))

print("Average of Cross Validation Scores: {}".format(score.mean()))

Cross Validation Scores are: [0.61452514 0.65730337 0.66292135 0.7247191  0.74719101]

Average of Cross Validation Scores: 0.6813319942250958


# Use random search and hyperopt packages to identify the best possible SVM model from the following search space.
# 'C': [0.1, 1, 10, 100, 1000]
# 'gamma': [1, 0.1, 0.01, 0.001, 0.0001] 
# 'kernel': ['rbf']

In [9]:
search_space ={'C': [0.1, 1, 10, 100, 1000],
     'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
     'kernel': ['rbf']}

# Random Search

In [10]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=search_space,verbose=2, n_jobs=-1,
                            n_iter=50)
random_result = random_search.fit(X_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 25 is smaller than n_iter=50. Running 25 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Score:  68.94809415936176
Best Params:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 100}


In [11]:
from sklearn.svm import SVC
model_svc = SVC(kernel='rbf', C=1000, gamma=0.001)
model_svc.fit(X_train,y_train)

y_pred = model_svc.predict(X_test)

print('Accuracy Score:\n',accuracy_score(y_test, y_pred))

Accuracy Score:
 0.7541899441340782


In [12]:
# Using 5-Fold Cross Validation to generalise the model accuracy.

from sklearn.model_selection import cross_val_score,KFold
kf=KFold(n_splits=5)
score=cross_val_score(model_svc,X,y,cv=kf)

print("Cross Validation Scores are: {}\n".format(score))
print("Average of Cross Validation Scores: {}".format(score.mean()))

Cross Validation Scores are: [0.63687151 0.64606742 0.73033708 0.71348315 0.74157303]

Average of Cross Validation Scores: 0.6936664365074383


# HypetOpt

In [16]:
pip install hyperopt


Note: you may need to restart the kernel to use updated packages.


In [17]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

def hyperparameter_tuning(params):
    clf = SVC(**params)
    acc = cross_val_score(clf, X, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [18]:
# Initialize trials object

search_space = {
    "C": hp.choice("C", [0.1, 1, 10, 100, 1000]),
    "gamma": hp.choice("gamma", [1, 0.1, 0.01, 0.001, 0.0001]),
    "kernel": hp.choice("kernel", ['rbf']),
}

trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = search_space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [00:40<00:00,  1.22trial/s, best loss: -0.7060134329295085]
Best: {'C': 4, 'gamma': 1, 'kernel': 0}


In [21]:
from sklearn.svm import SVC
model_svc = SVC(C= 4, gamma= 1, kernel= 'rbf')
model_svc.fit(X_train,y_train)

y_pred = model_svc.predict(X_test)

print('Accuracy Score:\n',accuracy_score(y_test, y_pred))

Accuracy Score:
 0.7262569832402235


In [22]:
# Use 5-Fold Cross Validation to generalise the model accuracy.

from sklearn.model_selection import cross_val_score,KFold

kf=KFold(n_splits=5)
score=cross_val_score(model_svc,X,y,cv=kf)

print("Cross Validation Scores are: {}\n".format(score))
print("Average of Cross Validation Scores: {}".format(score.mean()))

Cross Validation Scores are: [0.67039106 0.61797753 0.7247191  0.71348315 0.76404494]

Average of Cross Validation Scores: 0.6981231561107275
